In [1]:
#el dataset de crímenes pesa demasiado y no se carga en mysql por lo que procedemos a limpiar algo los datos antes en python

In [2]:
import pandas as pd 

In [3]:
df=pd.read_csv('NYPD_Complaint_Data_Historic.csv', dtype=str)

In [4]:
#cambiamos el tipo de data de la columna a fecha
df["CMPLNT_FR_DT"]=pd.to_datetime(df["CMPLNT_FR_DT"], errors='coerce')

In [5]:
#creamos una nueva lista con las columnas que queremos eliminar y las eliminamos del dataframe

columns_to_drop = ["CMPLNT_FR_TM", "CMPLNT_TO_DT", "CMPLNT_TO_TM", "ADDR_PCT_CD", "RPT_DT", "KY_CD", "OFNS_DESC", "PD_CD", "PD_DESC", "CRM_ATPT_CPTD_CD", "LOC_OF_OCCUR_DESC",\
                   "PREM_TYP_DESC", "JURIS_DESC", "JURISDICTION_CODE", "PARKS_NM", "HADEVELOPT", "HOUSING_PSA", "X_COORD_CD", "Y_COORD_CD", "SUSP_AGE_GROUP", "SUSP_RACE",\
                    "SUSP_SEX", "TRANSIT_DISTRICT", "Latitude", "Longitude", "Lat_Lon", "PATROL_BORO", "STATION_NAME", "VIC_AGE_GROUP", "VIC_RACE", "VIC_SEX"]

df = df.drop(columns = columns_to_drop, axis = 1)

In [6]:
#eliminamos las filas que no tienen barrio especificado

df = df.dropna(subset = ["BORO_NM"])

In [7]:
#eliminamos todas las filas duplicadas

df.drop_duplicates(keep="first", inplace=True)

df.duplicated().sum()

0

In [8]:
#filtramos los dos últimos años

df=df[df["CMPLNT_FR_DT"]>='2016-01-01']

In [36]:
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,LAW_CAT_CD,BORO_NM,anyo
2,802896158,2016-10-27,VIOLATION,BROOKLYN,2016
9,895680254,2016-09-17,MISDEMEANOR,BROOKLYN,2016
14,882568598,2017-06-28,VIOLATION,STATEN ISLAND,2017
29,409783199,2017-03-25,FELONY,BRONX,2017
30,791737161,2016-03-02,MISDEMEANOR,MANHATTAN,2016


In [35]:
#creamos una columna que tenga tan solo los años

df["anyo"] = pd.to_datetime(df["CMPLNT_FR_DT"]).dt.year

In [13]:
#agrupamos por barrio y delito y eliminamos la columna de fechas

crimenes_por_barrio = df.groupby(["BORO_NM", "LAW_CAT_CD"]).count().reset_index().drop(columns="CMPLNT_FR_DT")

In [72]:
#creamos columnas por cada uno de los delitos

crimenes_por_barrio = crimenes_por_barrio.pivot_table(index="BORO_NM", columns="LAW_CAT_CD", values="CMPLNT_NUM").reset_index()

In [38]:
#agrupamos las columnas de barrio, infracción y año y sumamos el número de infracciones

crimenes_por_barrio_y_anyo = df.groupby(["BORO_NM", "LAW_CAT_CD", "anyo"]).count().reset_index().drop(columns="CMPLNT_FR_DT")

In [42]:
#creamos nuevas columnas con cada una de las infracciones y los años

crimenes_por_barrio_y_anyo = crimenes_por_barrio_y_anyo.pivot_table(index="BORO_NM", columns=["LAW_CAT_CD", "anyo"], values="CMPLNT_NUM").reset_index()

In [51]:
#revisamos cada tipo de crimen y cada año para calcular la diferencia de un determinado año con respecto al anterior

for crimen in ["FELONY", "MISDEMEANOR", "VIOLATION"]:
    for anyo in range(2017, 2019):   
        crimenes_por_barrio_y_anyo[f"diferencia_{anyo}"] = crimenes_por_barrio_y_anyo[crimen][anyo] - crimenes_por_barrio_y_anyo[crimen][anyo-1]

In [63]:
# creamos una tabla con el barrio y la diferencia en la evolución de cada año

crimenes_por_barrio_y_anyo_2 = crimenes_por_barrio_y_anyo[["BORO_NM","diferencia_2017","diferencia_2018"]]

In [67]:
#eliminamos el multiindex
crimenes_por_barrio_y_anyo_2.columns = crimenes_por_barrio_y_anyo_2.columns.droplevel(level=1)

In [81]:
#creamos un nuevo dataframe con todas las columnas:barrio, infracciones y evolución en el nº de infracciones por año

crimenes = crimenes_por_barrio.merge(crimenes_por_barrio_y_anyo_2[["BORO_NM","diferencia_2017","diferencia_2018"]], how="left", left_on="BORO_NM", right_on="BORO_NM")

In [82]:
crimenes

LAW_CAT_CD,BORO_NM,FELONY,MISDEMEANOR,VIOLATION,diferencia_2017,diferencia_2018
0,BRONX,85756,176914,46515,530,893
1,BROOKLYN,133404,219192,61230,-670,1232
2,MANHATTAN,107301,195120,41327,-269,1248
3,QUEENS,89483,143452,42482,540,260
4,STATEN ISLAND,14711,35552,13533,250,-77


In [83]:
crimenes.to_csv('NYPD_Complaint_Data_Historic_3years.csv', index=False)